
### Steps:

step 1: clean up columns by addressing abbreviations

Step 2: create dataframe that contain all public health sentences

Step 3: for each row, extract sentences that contain at least one keywords that belong to public health keyword lists for each sentences, put a new row into the dataframe, 

step 4: save the new pandas dataframe into excel file

Step 5: repeat step 2, 3, and 4 for natural disaster keywords



In [81]:
import pandas as pd
import re

In [82]:
#data = pd.read_csv(r"C:\Users\ysxia\Dropbox\Code\MutualFund\Data\2021_07\2021_07_MF_data_all_columns_year2010_2020_de_dup.csv")
data = pd.read_csv(r"C:\Users\ysxia\Dropbox\Code\MutualFund\Data\2021_07\2021_07_MF_data_all_columns_year2010_2020.csv")

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20,21,24,36,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [83]:
data.shape

(164603, 49)

In [84]:
data.head(1)

,Unnamed: 0,SEC_film_#,SEC_fund_file#,accession#,class_#,class_#_2,class_#_3,class_#_4,class_#_5,class_#_6,...,principal_risks,principal_strategies,risks_wc,sec_act,series_cik1,stock_series#,strategies_wc,total_word_counts,crsp_class,filing_year
0,0,10638475.0,002-10758,0000004568-10-000004,c000023937,c000023938,c000023939,c000023940,c000023941,c000023942,...,you may lose money by investing in the fund. t...,the fund approaches the management of its inve...,NaN,1933 act,s000008801,s000008801,NaN,50077,M,2010


In [85]:
data['filing_year'].value_counts()

2017    17203
2020    16737
2016    16610
2018    16357
2015    16169
2019    15540
2013    15287
2014    15052
2012    13226
2011    12933
2010     9489
Name: filing_year, dtype: int64

In [86]:
data1 = data.dropna(subset=['principal_risks'])

In [87]:
data1['filing_year'].value_counts()

2017    17203
2020    16737
2016    16610
2018    16357
2015    16169
2019    15540
2013    15287
2014    15052
2012    13226
2011    12933
2010     9489
Name: filing_year, dtype: int64

### Extract Sentences

In [88]:
abbreviations = {'dr.': 'doctor', 'mr.': 'mister', 'bro.': 'brother', 'bro': 'brother', 'mrs.': 'mistress', 'ms.': 'miss', 'jr.': 'junior', 'sr.': 'senior',
                 'e.g.': 'for example', 'vs.': 'versus', 'U.S.': 'United States','etc.': 'etcetera', 'J.P.': 'Justice of the Peace',
                 'Inc.': 'Incorporated', 'LLC.': 'limited liability corporation', 'Co.': 'company', 'l.p.': 'limited partneship',
                 'ltd.': 'limited', 'Jan.': 'January', 'Feb.': 'February', 'Mar.': 'March', 'Apr.': 'April', 'i.e.': 'for example',
                 'Jun.': 'June', 'Jul.': 'July', 'Aug.': 'August', 'Oct.': 'October', 'Dec.': 'December', 'S.E.C.': 'SEC', 'Inv. Co. Act': 'Investment Company Act'}
terminators = ['.', '!', '?']
wrappers = ['"', "'", ')', ']', '}']

In [89]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1:
            return
        yield start
        start += len(sub)

In [90]:
def find_sentence_end(paragraph):
    [possible_endings, contraction_locations] = [[], []]
    contractions = abbreviations.keys()
    sentence_terminators = terminators + [terminator + wrapper for wrapper in wrappers for terminator in terminators]
    for sentence_terminator in sentence_terminators:
        t_indices = list(find_all(paragraph, sentence_terminator))
        possible_endings.extend(([] if not len(t_indices) else [[i, len(sentence_terminator)] for i in t_indices]))
    for contraction in contractions:
        c_indices = list(find_all(paragraph, contraction))
        contraction_locations.extend(([] if not len(c_indices) else [i + len(contraction) for i in c_indices]))
    possible_endings = [pe for pe in possible_endings if pe[0] + pe[1] not in contraction_locations]
    if len(paragraph) in [pe[0] + pe[1] for pe in possible_endings]:
        max_end_start = max([pe[0] for pe in possible_endings])
        possible_endings = [pe for pe in possible_endings if pe[0] != max_end_start]
    possible_endings = [pe[0] + pe[1] for pe in possible_endings if sum(pe) > len(paragraph) or (sum(pe) < len(paragraph) and paragraph[sum(pe)] == ' ')]
    end = (-1 if not len(possible_endings) else max(possible_endings))
    return end

In [91]:
def find_sentences(paragraph):
    replacingList = [["?", "? "], ["!", "! "],[".", ". "],["(", " "], [")", " "], [",", " "], ["i. e.", "i.e"], 
                     ["e. g.", "e.g."],["U. S.", "U.S."], ["J. P.", "J.P."], ["l. p.", "l.p."], 
                     ["S. E. C.", "S.E.C."]]
    for items in replacingList:
        paragraph = paragraph.replace(items[0], items[1])
    paragraph = re.sub(' +', ' ', paragraph)
    if paragraph != "":
        if paragraph[0] == '?' or paragraph[0] == '.' or paragraph[0] == '!':
            paragraph = paragraph[1:]
    end = True
    sentences = []
    while end > -1:
        end = find_sentence_end(paragraph)
        if end > -1:
            sentences.append(paragraph[end:].strip())
            paragraph = paragraph[:end]
    sentences.append(paragraph)
    sentences.reverse()
    return sentences

### input paragraphs, then split into sentences by using the above function find_sentences()

In [92]:
data1['cleaned_principal_risks']= ""
for row in range(len(data1)):
    if row %5000 == 0:
        print ("processing row: ", row)
    para = data1['principal_risks'].iloc[row]
    sents = find_sentences(para)
    data1['cleaned_principal_risks'].iloc[row] = sents

processing row:  0


C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


processing row:  5000
processing row:  10000
processing row:  15000
processing row:  20000
processing row:  25000
processing row:  30000
processing row:  35000
processing row:  40000
processing row:  45000
processing row:  50000
processing row:  55000
processing row:  60000
processing row:  65000
processing row:  70000
processing row:  75000
processing row:  80000
processing row:  85000
processing row:  90000
processing row:  95000
processing row:  100000
processing row:  105000
processing row:  110000
processing row:  115000
processing row:  120000
processing row:  125000
processing row:  130000
processing row:  135000
processing row:  140000
processing row:  145000
processing row:  150000
processing row:  155000
processing row:  160000


In [93]:
data1.shape

(164603, 50)

In [94]:
data1.head(1)

,Unnamed: 0,SEC_film_#,SEC_fund_file#,accession#,class_#,class_#_2,class_#_3,class_#_4,class_#_5,class_#_6,...,principal_strategies,risks_wc,sec_act,series_cik1,stock_series#,strategies_wc,total_word_counts,crsp_class,filing_year,cleaned_principal_risks
0,0,10638475.0,002-10758,0000004568-10-000004,c000023937,c000023938,c000023939,c000023940,c000023941,c000023942,...,the fund approaches the management of its inve...,NaN,1933 act,s000008801,s000008801,NaN,50077,M,2010,"[you may lose money by investing in the fund.,..."


In [95]:
type(data1['cleaned_principal_risks'].iloc[0])

list

In [96]:
data1.to_csv('2021_07_principal_risks_cleaned_non_dedup.csv')

### Public Health Keywords to Search Sentences that Contain one or multiple of these keywords

In [65]:
# Public Health keywords: 
ph_word = ['communicable diseases','health crises','pandemic','respiratory','illness','prevention','epidemic',
           'coronavirus','health crisis','pandemics','sanitation','global health crises',
           'covid','health screenings','pathogens','sars','global health crisis',
           'covid 19','hiv','preparedness','sars cov 2','epidemics','disease','influenza',
           'public health','virus','global health','diseases','mers','quarantines','h1n1','viruses']

ph_word = [ 'communicable   diseases',         
            'coronavirus',                          
            'covid','covid 19',                     
            'disease',                              
            'epidemic',                             
            'global health',                        
            'global health crisis',                 
            'health crises',                        
            'health crisis' ,                       
            'health screenings' ,                   
            'hiv' ,                                 
            'influenza', 'h1n1',                     
            'pandemic','epidemic',             
            'pathogens',                            
            'prevention',                           
            'public health',                        
            'quarantines',                          
            'respiratory illness',                  
            'sanitation',                           
            'sars', 'sars cov 2', 'mers',        
            'virus']                                


In [66]:
import os
import pandas as pd
import glob
import nltk.data
import re
#from nltk.tokenize import tokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters, PunktTrainer

pubic_health = pd.DataFrame()
ph_sents = []
for row in range(len(data1)):
    if row %5000 == 0:
        print ("processing row: ", row)
    sentences = data1['cleaned_principal_risks'].iloc[row]
    for i in range(len(sentences)):
        for word in ph_word:
            tokens = sentences[i].lower().split()
            if word in tokens:
                ph_sents.append([data1['accession#'].iloc[row],
                                 data1['filing_year'].iloc[row],
                                 data1['fund_name'].iloc[row],
                                 sentences[i]])
        
        
        #pubic_health = pubic_health.append({'accession#':data1['accession#'].iloc[row],
        #                                    'filing_year':data1['filing_year'].iloc[row],
        #                                    'fund_name':data1['fund_name'].iloc[row],                                                    
        #                                    'sentences':sentences[i]},ignore_index=True)
                                            #'sentences':re.sub(' +', ' '," ".join(sentences[i]).replace('\n',' '))},ignore_index=True)
pubic_health = pd.DataFrame(ph_sents, columns =['accession#',
                                                'filing_year',
                                                'fund_name', 
                                                'sentences'])               

processing row:  0
processing row:  5000
processing row:  10000
processing row:  15000
processing row:  20000
processing row:  25000
processing row:  30000
processing row:  35000
processing row:  40000
processing row:  45000
processing row:  50000
processing row:  55000
processing row:  60000
processing row:  65000
processing row:  70000
processing row:  75000
processing row:  80000
processing row:  85000
processing row:  90000
processing row:  95000


In [67]:
pubic_health.shape

(11051, 4)

In [68]:
pubic_health.head()

,accession#,filing_year,fund_name,sentences
0,0000318478-20-000025,2020,"bny mellon appreciation fund, inc.",&nbsp; recent examples include pandemic risks ...
1,0000318478-20-000025,2020,"bny mellon appreciation fund, inc.",&nbsp; recent examples include pandemic risks ...
2,0000885409-20-000004,2020,"dreyfus basic money market fund, inc",&nbsp; recent examples include pandemic risks ...
3,0000885409-20-000004,2020,"dreyfus basic money market fund, inc",&nbsp; recent examples include pandemic risks ...
4,0000075176-20-000010,2020,bny mellon municipal bond fund,primarily in response to the covid-19 pandemic...


In [75]:
pubic_health['filing_year'].value_counts()

2020    9109
2017     297
2018     278
2016     268
2015     247
2019     242
2014     180
2013     132
2012     130
2011     112
2010      56
Name: filing_year, dtype: int64

In [69]:
pubic_health.to_excel("2021_07_public_health_keywords_sentences_dedup.xlsx")

In [70]:
 nd_word = [ 'cyclones',                   
             'death',                      
             'drought',                    
             'earthquakes',                
             'environmental damage',       
             'cloud',                      
             'disaster',                   
             'droughts',                   
             'earthquake',                 
             'fire',                       
             'flood',                      
             'hurricane',                  
             'hurricanes',                 
             'lightning',                  
             'natural disaster',           
             'natural disasters',          
             'seismic',                    
             'storms',                     
             'tornadoes',                  
             'tsunami',                    
             'underground',                
             'volcanoes',                  
             'windstorms']


In [71]:
nd_sents = []
for row in range(len(data1)):
    if row %5000 == 0:
        print ("processing row: ", row)
    sentences = data1['cleaned_principal_risks'].iloc[row]
    for i in range(len(sentences)):
        for word in nd_word:
            tokens = sentences[i].lower().split()
            if word in tokens:
                nd_sents.append([data1['accession#'].iloc[row],
                                 data1['filing_year'].iloc[row],
                                 data1['fund_name'].iloc[row],
                                 sentences[i]])
        
        
        #pubic_health = pubic_health.append({'accession#':data1['accession#'].iloc[row],
        #                                    'filing_year':data1['filing_year'].iloc[row],
        #                                    'fund_name':data1['fund_name'].iloc[row],                                                    
        #                                    'sentences':sentences[i]},ignore_index=True)
                                            #'sentences':re.sub(' +', ' '," ".join(sentences[i]).replace('\n',' '))},ignore_index=True)
natural_disaster = pd.DataFrame(nd_sents, columns =['accession#',
                                                'filing_year',
                                                'fund_name', 
                                                'sentences'])   

processing row:  0
processing row:  5000
processing row:  10000
processing row:  15000
processing row:  20000
processing row:  25000
processing row:  30000
processing row:  35000
processing row:  40000
processing row:  45000
processing row:  50000
processing row:  55000
processing row:  60000
processing row:  65000
processing row:  70000
processing row:  75000
processing row:  80000
processing row:  85000
processing row:  90000
processing row:  95000


In [72]:
natural_disaster.shape

(8897, 4)

In [74]:
natural_disaster['filing_year'].value_counts()

2020    2138
2019    1075
2018    1047
2017     949
2016     863
2015     808
2014     598
2013     519
2012     453
2011     303
2010     144
Name: filing_year, dtype: int64

In [73]:
natural_disaster.to_excel("2021_07_natural_disaster_keywords_sentences_dedup.xlsx")

In [77]:
pubic_health_non_dedup = pd.read_excel('2021_07_public_health_keywords_sentences.xlsx')

In [78]:
pubic_health_non_dedup.head(2)

,Unnamed: 0,accession#,filing_year,fund_name,sentences
0,0,0000067590-10-000940,2010,prudential real assets fund,prices of commodities and related contracts ma...
1,1,0000088053-10-000446,2010,dws enhanced commodity strategy fund,the value of commodity-linked derivative instr...


In [80]:
pubic_health_non_dedup['filing_year'].value_counts()

2020    14045
2017      624
2016      524
2018      464
2015      425
2019      393
2014      289
2012      239
2013      239
2011      192
2010      102
Name: filing_year, dtype: int64